In [1]:
#include <iostream>
#include <string>
#include <math.h>       /* cosh, log */
using namespace std;

In [2]:
/*Cargo la librería de Delphes*/
gSystem->Load("/home/camilo/Proyectos/Delphes/delphes/libDelphes.so");

Error in <TCling::RegisterModule>: cannot find dictionary module libClassesDict_rdict.pcm
Error in <TCling::RegisterModule>: cannot find dictionary module libDisplayDict_rdict.pcm
Error in <TCling::RegisterModule>: cannot find dictionary module libFastJetDict_rdict.pcm
Error in <TCling::RegisterModule>: cannot find dictionary module libModulesDict_rdict.pcm
Error in <TCling::RegisterModule>: cannot find dictionary module libExRootAnalysisDict_rdict.pcm


In [3]:
/*Instancio un objeto TChain*/
TChain chain("Delphes");

In [4]:
/*Creo las variables necesarias para el for*/
std::string nom1="/Storage/scratch/camilo/Proyectos/MadGraph/Datos/Signal_prueba-2/Events/run_";
std::string nom2="/tag_1_delphes_events.root";
std::string num, out;

In [5]:
/*Llenado del chain con todos los runs*/
for ( i=1; i<101; i=i+1 ) {
   num = std::to_string(i);
   if( i < 10 ) {
       out= nom1+"0"+num+nom2;
    }
//convierto en cadena de caracteres constante (es lo que recibe la función)
    const char * c = out.c_str();
/*pego todos los datos*/
    chain.Add(c);
}

In [6]:
//Creo un objeto de treereader, que recive la dirección en memoria de
//la cadena (TChain) como argumento
ExRootTreeReader *treeReader = new ExRootTreeReader(&chain);

In [7]:
/*Leo el numero de entradas*/
Long64_t numberOfEntries = treeReader->GetEntries()

(long long) 39581


In [8]:
/*Obtengo las ramas que deseo*/
TClonesArray *branchMuon = treeReader->UseBranch("Muon");
TClonesArray *branchMissingET = treeReader->UseBranch("MissingET");
TClonesArray *branchJet = treeReader->UseBranch("Jet");

In [9]:
// Inicializo los contadores
Int_t cut1=0, cut2=0, cut3=0, cut4=0, cut5=0;

In [10]:
for(Int_t entry = 0; entry < numberOfEntries; ++entry)
    {
        // Se carga un evento espesifico para ser analizado
        treeReader->ReadEntry(entry);

        // Se hace el primer corte, se pide que tenga al menos dos jets
        if(branchJet->GetEntries() > 1)
        {
            ++cut1; //Cuantos eventos pasan el corte 1
            // se toma el primer jet y se hace un cast a la clase Jet
            Jet *jet1 = (Jet*) branchJet->At(0);
            Jet *jet2 = (Jet*) branchJet->At(1);

            if( jet1->BTag>0 && jet2->BTag>0 )
            {
                ++cut2; //Cuantos eventos pasan el corte 2
                //Inicializo los obejos clase muon
                Muon *muon1, *muon2;

                // Para ser contados deben de tener al menos 2 muones
                if(branchMuon->GetEntries() > 1)
                {
                    ++cut3; //Cuantos eventos pasan el corte 3
                    // Take first two electrons, tomo solo dos porque estan organizados por PT
                    muon1 = (Muon *) branchMuon->At(0);
                    muon2 = (Muon *) branchMuon->At(1);
                    //Si la carga es diferente
                    int charge = muon1->Charge * muon2->Charge;
                    if( charge < 0 )
                    {
                        ++cut4; //Cuantos eventos pasan el corte 4
                
                        double MassMuon=(muon1->P4()+muon2->P4()).M();
                        if( !((MassMuon>80)&&(MassMuon<100)) )
                        {
                            ++cut5; //Cuantos eventos pasan el corte 4
                     
                            
                            // Plot jet transverse momentum
                            //histJetPT->Fill(jet->PT);
                            // Plot their invariant mass
                            //histMass->Fill(((elec1->P4()) + (elec2->P4())).M());
               
                        
                        
                        }
                    }
                }
            }
        }
    }

In [11]:
cout<<"#Raw ="<<numberOfEntries<<endl;
cout<<"#Cut1="<<cut1<<endl;
cout<<"#Cut2="<<cut2<<endl;
cout<<"#Cut3="<<cut3<<endl;
cout<<"#Cut4="<<cut4<<endl;
cout<<"#Cut5="<<cut5<<endl<<endl;


float EFFN1=(float)cut1/(float)numberOfEntries;
float EFFN2=(float)cut2/(float)cut1;
float EFFN3=(float)cut3/(float)cut2;
float EFFN4=(float)cut4/(float)cut3;
float EFFN5=(float)cut5/(float)cut4;


float EFFA1=(float)cut1/(float)numberOfEntries;
float EFFA2=(float)cut2/(float)numberOfEntries;
float EFFA3=(float)cut3/(float)numberOfEntries;
float EFFA4=(float)cut4/(float)numberOfEntries;
float EFFA5=(float)cut5/(float)numberOfEntries;

cout<<"EFFA1="<<EFFA1<<endl;
cout<<"EFFA2="<<EFFA2<<endl;
cout<<"EFFA3="<<EFFA3<<endl;
cout<<"EFFA4="<<EFFA4<<endl;
cout<<"EFFA5="<<EFFA5<<endl<<endl;


cout<<"EFFN1="<<EFFN1<<endl;
cout<<"EFFN2="<<EFFN2<<endl;
cout<<"EFFN3="<<EFFN3<<endl;
cout<<"EFFN4="<<EFFN4<<endl;
cout<<"EFFN5="<<EFFN5<<endl;

#Raw =39581
#Cut1=37329
#Cut2=7475
#Cut3=4149
#Cut4=4149
#Cut5=3832
EFFA1=0.943104
EFFA2=0.188853
EFFA3=0.104823
EFFA4=0.104823
EFFA5=0.0968141
EFFN1=0.943104
EFFN2=0.200246
EFFN3=0.55505
EFFN4=1
EFFN5=0.923596


In [12]:
/*Instancio un objeto TChain*/
TChain chainB1("Delphes");
/*Creo las variables necesarias para el for*/
std::string nomB1="/Storage/scratch/camilo/Proyectos/MadGraph/Datos/Signal_prueba-2/Events/run_";
std::string nom2B1="/tag_1_delphes_events.root";
std::string numB1, outB1;
/*Llenado del chain con todos los runs*/
for ( i=1; i<101; i=i+1 ) {
   numB1 = std::to_string(i);
   if( i < 10 ) {
       outB1= nomB1+"0"+numB1+nom2B1;
    }
//convierto en cadena de caracteres constante (es lo que recibe la función)
    const char * cB1 = outB1.c_str();
/*pego todos los datos*/
    chainB1.Add(cB1);
}
//Creo un objeto de treereader, que recive la dirección en memoria de
//la cadena (TChain) como argumento
ExRootTreeReader *treeReaderB1 = new ExRootTreeReader(&chainB1);
/*Leo el numero de entradas*/
Long64_t numberOfEntriesB1 = treeReaderB1->GetEntries();
/*Obtengo las ramas que deseo*/
TClonesArray *branchMuonB1 = treeReaderB1->UseBranch("Muon");
TClonesArray *branchMissingETB1 = treeReaderB1->UseBranch("MissingET");
TClonesArray *branchJetB1 = treeReaderB1->UseBranch("Jet");
// Inicializo los contadores
Int_t cut1B1=0, cut2B1=0, cut3B1=0, cut4B1=0, cut5B1=0;
for(Int_t entryB1 = 0; entryB1 < numberOfEntriesB1; ++entryB1)
    {
        // Se carga un evento espesifico para ser analizado
        treeReaderB1->ReadEntry(entryB1);

        // Se hace el primer corte, se pide que tenga al menos dos jets
        if(branchJetB1->GetEntries() > 1)
        {
            ++cut1B1; //Cuantos eventos pasan el corte 1
            // se toma el primer jet y se hace un cast a la clase Jet
            Jet *jet1B1 = (Jet*) branchJetB1->At(0);
            Jet *jet2B1 = (Jet*) branchJetB1->At(1);

            if( jet1B1->BTag>0 && jet2B1->BTag>0 )
            {
                ++cut2B1; //Cuantos eventos pasan el corte 2
                //Inicializo los obejos clase muon
                Muon *muon1B1, *muon2B1;

                // Para ser contados deben de tener al menos 2 muones
                if(branchMuonB1->GetEntries() > 1)
                {
                    ++cut3B1; //Cuantos eventos pasan el corte 3
                    // Take first two electrons, tomo solo dos porque estan organizados por PT
                    muon1B1 = (Muon *) branchMuonB1->At(0);
                    muon2B1 = (Muon *) branchMuonB1->At(1);
                    //Si la carga es diferente
                    int chargeB1 = muon1B1->Charge * muon2B1->Charge;
                    if( chargeB1 < 0 )
                    {
                        ++cut4B1; //Cuantos eventos pasan el corte 4
                
                        double MassMuonB1=(muon1B1->P4()+muon2B1->P4()).M();
                        if( !((MassMuonB1>80)&&(MassMuonB1<100)) )
                        {
                            ++cut5B1; //Cuantos eventos pasan el corte 4
                     
                            
                            // Plot jet transverse momentum
                            //histJetPT->Fill(jet->PT);
                            // Plot their invariant mass
                            //histMass->Fill(((elec1->P4()) + (elec2->P4())).M());
               
                        
                        
                        }
                    }
                }
            }
        }
    }
cout<<"#Raw ="<<numberOfEntriesB1<<endl;
cout<<"#Cut1="<<cut1B1<<endl;
cout<<"#Cut2="<<cut2B1<<endl;
cout<<"#Cut3="<<cut3B1<<endl;
cout<<"#Cut4="<<cut4B1<<endl;
cout<<"#Cut5="<<cut5B1<<endl<<endl;


float EFFN1B1=(float)cut1B1/(float)numberOfEntriesB1;
float EFFN2B1=(float)cut2B1/(float)cut1B1;
float EFFN3B1=(float)cut3B1/(float)cut2B1;
float EFFN4B1=(float)cut4B1/(float)cut3B1;
float EFFN5B1=(float)cut5B1/(float)cut4B1;


float EFFA1B1=(float)cut1B1/(float)numberOfEntriesB1;
float EFFA2B1=(float)cut2B1/(float)numberOfEntriesB1;
float EFFA3B1=(float)cut3B1/(float)numberOfEntriesB1;
float EFFA4B1=(float)cut4B1/(float)numberOfEntriesB1;
float EFFA5B1=(float)cut5B1/(float)numberOfEntriesB1;

cout<<"EFFA1="<<EFFA1B1<<endl;
cout<<"EFFA2="<<EFFA2B1<<endl;
cout<<"EFFA3="<<EFFA3B1<<endl;
cout<<"EFFA4="<<EFFA4B1<<endl;
cout<<"EFFA5="<<EFFA5B1<<endl<<endl;


cout<<"EFFN1="<<EFFN1B1<<endl;
cout<<"EFFN2="<<EFFN2B1<<endl;
cout<<"EFFN3="<<EFFN3B1<<endl;
cout<<"EFFN4="<<EFFN4B1<<endl;
cout<<"EFFN5="<<EFFN5B1<<endl;

#Raw =39581
#Cut1=37329
#Cut2=7475
#Cut3=4149
#Cut4=4149
#Cut5=3832
EFFA1=0.943104
EFFA2=0.188853
EFFA3=0.104823
EFFA4=0.104823
EFFA5=0.0968141
EFFN1=0.943104
EFFN2=0.200246
EFFN3=0.55505
EFFN4=1
EFFN5=0.923596
